In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

from helpers import savefig

# Comparison of angle and distance-based point recovery

Compare reconstruction from distances with reconstruction from angles.

*The data is obtained with the script simulation_angles_distances.py*.

In [ ]:
#fname = 'results/angles_distances_ICASSP.pkl'
fname = 'results/angles_distances.pkl'

df_results = pd.read_pickle(fname)
df_results.head()
df_results.loc[:, 'square_size'] = df_results.loc[:, 'size']
df_results.loc[:, 'n_it'] = df_results.n_it.astype(np.float16)

# choose parameters to plot (needs to be adjusted for new data)
plot_sizes = [1, 2, 3, 4, 5]
plot_sigmas = [1e-3, 1e-2, 1e-1]

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(6, 2.5)

delta = 1
colors = sns.color_palette('Blues', n_colors=len(plot_sizes) + delta)
max_sigma = 10

for i, size in enumerate(plot_sizes):
    data_size = df_results[df_results.square_size == size]
    error_angle = data_size.loc[data_size.type == 'angle']

    df = error_angle[['sigma', 'error']].astype(np.float32)
    df_means = df.groupby('sigma').agg('median')

    sigmas = df_means.index.values
    xs = sigmas[sigmas <= max_sigma]
    ys = df_means.error.values[sigmas <= max_sigma]
    ax.loglog(xs, ys, color=colors[i + delta], label=size)

palettes = ['Greens', 'Oranges', 'Purples']
linestyles = [':', '-.', '--']
handles2 = []
for j, sigma_distance in enumerate(plot_sigmas):
    data = df_results[(df_results.type == 'angle') | ((df_results.type == 'distance') &
                                                      (df_results.sigma == sigma_distance))]
    colors_dist = sns.color_palette(palettes[j], n_colors=len(plot_sizes) + delta)
    for i, size in enumerate(plot_sizes):
        data_size = data[data.square_size == size]
        error_distance = data_size.loc[data_size.type == 'distance', 'error'].mean()
        ax.axhline(y=error_distance, color=colors_dist[i + delta], ls=linestyles[j])
    h2 = ax.axhline(y=error_distance, color=colors_dist[i + delta], ls=linestyles[j])
    handles2.append(h2)

ax.set_xlabel('angle noise $\sigma_a$')
ax.set_ylabel('accuracy')
ax.grid()
handels, labels = ax.get_legend_handles_labels()
l1 = ax.legend(handels[:1] + handels[2:3] + handels[-1:],
               labels[:1] + labels[2:3] + labels[-1:],
               loc='lower right',
               title='square side',
               framealpha=1.0)
ax.legend(handles2, ['{:1.0e}'.format(s) for s in plot_sigmas],
          loc='upper left',
          title='distance noise $\sigma_d$',
          framealpha=1.0)
ax.add_artist(l1)
ax.set_xlim(1e-5, 2)
#savefig('plots/comparison.pdf', fig=fig)